# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="./Banana_Windows_x86_64/Banana.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    print("The action is {}".format(action))
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

In [15]:
def dqn(agent, env, num_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    
    eps=eps_start
    score = 0
    score_window = deque(maxlen=100)
    scores = []
    
    for i_episode in range(1, num_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps).astype(int)
            
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0] 
            
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
                
            score_window.append(score)
            scores.append(score)
            
            eps = max(eps_end, eps_decay*eps)
            
        #print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(score_window)), end="")
            
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(score_window)))
            
    
                

In [ ]:
from agent import Agent
env_info = env.reset(train_mode=False)[brain_name]
# Get the environment params
state_size = len(env_info.vector_observations[0])
num_actions = brain.vector_action_space_size

agent = Agent(state_size, num_actions)


dqn(agent, env)


THE DEVICE IS
cuda:0
QNet(
  (h1): Linear(in_features=37, out_features=10, bias=True)
  (h2): Linear(in_features=10, out_features=10, bias=True)
  (out): Linear(in_features=10, out_features=4, bias=True)
)
[0.         0.         0.         1.         0.         0.
 1.         0.         0.         0.84109235 0.         1.
 0.         0.         0.30561668 0.         0.         0.
 1.         0.         0.         0.         1.         0.
 0.30040157 1.         0.         0.         0.         0.6890496
 0.         0.         1.         0.         0.33517891 0.
 0.        ]
[0.         0.         1.         0.         0.27476281 0.
 1.         0.         0.         0.13858734 0.         1.
 0.         0.         0.20260113 0.         1.         0.
 0.         0.30814925 0.         1.         0.         0.
 0.14369234 0.         1.         0.         0.         0.17000253
 0.         1.         0.         0.         0.13022949 0.
 0.        ]
[1.         0.         0.         0.         

[0.         1.         0.         0.         0.2493148  0.
 1.         0.         0.         0.94481653 0.         0.
 0.         1.         0.         0.         1.         0.
 0.         0.53195781 0.         0.         1.         0.
 0.58050323 0.         0.         1.         0.         0.29130021
 0.         0.         1.         0.         0.23496012 0.
 0.        ]
[1.         0.         0.         0.         0.05674512 1.
 0.         0.         0.         0.22109106 1.         0.
 0.         0.         0.70194376 1.         0.         0.
 0.         0.04508644 0.         0.         1.         0.
 0.9867838  0.         0.         1.         0.         0.11210422
 1.         0.         0.         0.         0.40913516 0.
 0.        ]
[0.         1.         0.         0.         0.67104834 0.
 0.         1.         0.         0.27151158 0.         0.
 1.         0.         0.16752143 1.         0.         0.
 0.         0.26256379 0.         0.         1.         0.
 0.33303407 0.

[0.         0.         1.         0.         0.12982364 0.
 1.         0.         0.         0.37744132 0.         1.
 0.         0.         0.26410854 0.         1.         0.
 0.         0.39134467 0.         1.         0.         0.
 0.29141149 1.         0.         0.         0.         0.06168924
 0.         1.         0.         0.         0.41087982 0.
 0.        ]
[1.         0.         0.         0.         0.40311471 0.
 0.         1.         0.         0.75012624 0.         0.
 1.         0.         0.23144966 0.         0.         1.
 0.         0.28213298 1.         0.         0.         0.
 0.28567401 1.         0.         0.         0.         0.75093484
 1.         0.         0.         0.         0.5195505  0.
 0.        ]
[0.         0.         1.         0.         0.12122991 0.
 0.         1.         0.         0.18517667 0.         1.
 0.         0.         0.19877972 0.         0.         1.
 0.         0.0586048  0.         1.         0.         0.
 0.30470252 0.

[0.         1.         0.         0.         0.64798349 0.
 0.         1.         0.         0.61749339 0.         0.
 1.         0.         0.26124984 1.         0.         0.
 0.         0.05304393 0.         0.         1.         0.
 0.11186895 1.         0.         0.         0.         0.08311214
 0.         0.         1.         0.         0.09620027 0.
 0.        ]
[1.         0.         0.         0.         0.58631539 1.
 0.         0.         0.         0.07492492 0.         1.
 0.         0.         0.92661369 0.         0.         1.
 0.         0.55483335 0.         0.         1.         0.
 0.88070887 1.         0.         0.         0.         0.0885005
 1.         0.         0.         0.         0.09997048 0.
 0.        ]
[1.         0.         0.         0.         0.0946969  0.
 0.         1.         0.         0.21508326 0.         1.
 0.         0.         0.11867812 1.         0.         0.
 0.         0.08979848 0.         1.         0.         0.
 0.15599358 1. 

[0.         0.         1.         0.         0.15844744 1.
 0.         0.         0.         0.0559766  0.         0.
 0.         1.         0.         0.         0.         1.
 0.         0.61602473 0.         0.         1.         0.
 0.62630832 1.         0.         0.         0.         0.05481696
 1.         0.         0.         0.         0.41047645 0.
 0.        ]
[1.         0.         0.         0.         0.09593101 1.
 0.         0.         0.         0.18527989 0.         0.
 1.         0.         0.01688235 0.         0.         0.
 1.         0.         0.         0.         1.         0.
 0.0177338  1.         0.         0.         0.         0.29562095
 0.         0.         1.         0.         0.02332488 0.
 0.        ]
[1.         0.         0.         0.         0.20980482 0.
 0.         1.         0.         0.55134219 0.         1.
 0.         0.         0.25885254 0.         0.         1.
 0.         0.49427003 0.         1.         0.         0.
 0.38517052 0.

...

[1.         0.         0.         0.         0.68335825 1.
 0.         0.         0.         0.07518348 1.         0.
 0.         0.         0.61423153 0.         0.         1.
 0.         0.40907642 0.         0.         1.         0.
 0.83748645 0.         0.         0.         1.         0.
 1.         0.         0.         0.         0.09395323 0.
 0.        ]
[1.         0.         0.         0.         0.25646511 0.
 0.         1.         0.         0.27252555 0.         0.
 1.         0.         0.22310461 0.         0.         1.
 0.         0.26440123 0.         0.         1.         0.
 0.31330603 1.         0.         0.         0.         0.28482455
 0.         0.         1.         0.         0.20742653 0.
 0.        ]
[0.         1.         0.         0.         0.34054759 0.
 1.         0.         0.         0.22045177 1.         0.
 0.         0.         0.92570913 0.         1.         0.
 0.         0.23530924 0.         0.         1.         0.
 0.21569094 0.        

[1.         0.         0.         0.         0.16610041 0.
 1.         0.         0.         0.12247224 0.         1.
 0.         0.         0.20624822 0.         1.         0.
 0.         0.23659821 0.         1.         0.         0.
 0.13660003 0.         1.         0.         0.         0.14441654
 0.         1.         0.         0.         0.11875097 0.
 0.        ]
[0.         0.         1.         0.         0.15255289 1.
 0.         0.         0.         0.62364572 0.         1.
 0.         0.         0.27088106 1.         0.         0.
 0.         0.21995184 0.         0.         1.         0.
 0.16825622 1.         0.         0.         0.         0.33390716
 1.         0.         0.         0.         0.31349775 0.
 0.        ]
[0.         1.         0.         0.         0.20868921 0.
 1.         0.         0.         0.2674796  0.         1.
 0.         0.         0.50625807 0.         1.         0.
 0.         0.18913665 1.         0.         0.         0.
 0.34427592 0.

[0.         1.         0.         0.         0.36138022 0.
 0.         1.         0.         0.1508798  0.         0.
 1.         0.         0.07251114 0.         1.         0.
 0.         0.38614777 0.         1.         0.         0.
 0.33975971 0.         1.         0.         0.         0.4409501
 0.         0.         1.         0.         0.12794012 0.
 0.        ]
[0.         0.         0.         1.         0.         0.
 0.         1.         0.         0.35905993 0.         0.
 1.         0.         0.02860785 1.         0.         0.
 0.         0.68461245 0.         0.         1.         0.
 0.15225324 0.         0.         1.         0.         0.14289033
 0.         0.         1.         0.         0.41181764 0.
 0.        ]
[1.         0.         0.         0.         0.78296936 0.
 1.         0.         0.         0.34971568 0.         1.
 0.         0.         0.0850699  0.         0.         1.
 0.         0.44122776 0.         1.         0.         0.
 0.09870172 1. 

[0.         1.         0.         0.         0.28382099 1.
 0.         0.         0.         0.32351783 0.         0.
 1.         0.         0.56294149 0.         1.         0.
 0.         0.38143894 1.         0.         0.         0.
 0.7527566  0.         1.         0.         0.         0.81386858
 1.         0.         0.         0.         0.40111172 0.
 0.        ]
[0.         1.         0.         0.         0.32689449 0.
 1.         0.         0.         0.36055425 0.         0.
 1.         0.         0.15306932 0.         1.         0.
 0.         0.28174669 0.         0.         1.         0.
 0.46583912 0.         0.         1.         0.         0.09108429
 1.         0.         0.         0.         0.31128171 0.
 0.        ]
[0.         1.         0.         0.         0.09267785 0.
 1.         0.         0.         0.16081202 0.         0.
 0.         1.         0.         0.         1.         0.
 0.         0.09160803 0.         0.         1.         0.
 0.14749284 0.

[0.         0.         1.         0.         0.24764632 0.
 1.         0.         0.         0.92349046 0.         0.
 0.         1.         0.         0.         0.         1.
 0.         0.50766557 0.         0.         0.         1.
 0.         0.         0.         1.         0.         0.31135869
 1.         0.         0.         0.         0.15607964 0.
 0.        ]
[0.         0.         1.         0.         0.40142044 1.
 0.         0.         0.         0.36844757 0.         0.
 1.         0.         0.55975837 0.         0.         1.
 0.         0.14925577 1.         0.         0.         0.
 0.62566423 1.         0.         0.         0.         0.52629477
 0.         0.         1.         0.         0.21087211 0.
 0.        ]
[0.         0.         1.         0.         0.11925343 0.
 0.         1.         0.         0.24496296 1.         0.
 0.         0.         0.28592467 0.         0.         1.
 0.         0.12457043 1.         0.         0.         0.
 0.20074295 1.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```